Import files in s3 to pandas.

Assumes conda environment is configured as per venvs/shared/requirements.txt

Steps
1. Create file list from buckets

2. Filter files for tap_on and tap_off files

3. Create df_on and df_off as dataframes as concatenation of all tap_on and tap_off files, respectively

4. write to dask array 


In [ ]:
#Configure environment if needed
#mkdir datathon
#cd datathon
#git clone https://github.com/ned2/leastsquares.git

#!pip install -r /venvs/shared/requirements.txt



In [3]:
import pandas as pd
import boto3
import io
import gzip


In [4]:
bucket_name='datathon-2018'


In [5]:
#create file list you wish to read from
s3 = boto3.resource('s3')
bucket=s3.Bucket(bucket_name)
file_list_on=[]
#!for obj in bucket.objects.filter(Prefix='raw/MelbDatathon2018/Samp_0/'):
#    if(obj.key.endswith(".txt.gz")):
#        file_list.append(obj.key)
#                               

file_list=[obj.key for obj in bucket.objects.filter(Prefix='raw/MelbDatathon2018/Samp_0/') if obj.key.endswith(".txt.gz")]


In [8]:
# tap off
import re
tap_off_files = []
for f in file_list:
    if sum([bool(re.search(pattern='ScanOffTransaction', string=f))]):
        tap_off_files.append(f)

        

In [9]:
# tap on        
tap_on_files = []
for f in file_list:
    if sum([bool(re.search(pattern='ScanOnTransaction', string=f))]):
        tap_on_files.append(f)
        

In [ ]:
%%time
bucket_name='s3://datathon-2018/'
df_on = pd.concat([pd.read_csv(bucket_name+f, sep='|', compression='gzip', header=None) for f in tap_on_files])

In [ ]:
%%time
df_off = pd.concat([pd.read_csv(bucket_name+f, sep='|', compression='gzip', header=None) for f in tap_off_files])

In [ ]:
#to dask dataframe
from dask import dataframe as dd
sd_on = dd.from_pandas(df_on, npartitions=3)
sd_off = dd.from_pandas(df_off, npartitions=3)

In [ ]:
# to parquety
df_on.to_parquet('df_on.parquet.gzip', compression='gzip')
df_off.to_parquet('df_off.parquet.gzip', compression='gzip')


In [ ]:

# s3FS not working
#fs = s3fs.S3FileSystem(anon=True)
#fs.ls('datathon-ilana1')
    
    

In [ ]:
#s3_client=boto3.client('s3')
#obj = s3_client.get_object(Bucket='datathon-2018', Key=key)
#gz = gzip.GzipFile(fileobj=obj['Body'])
